## 교차 검증과 그리드 서치
- 훈련 세트 60%, 검증 세트 20%, 테스트 세트 20%

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine[['class']]

In [6]:
# test_size= : test set를 몇퍼센트로 할지(20퍼면 0.2)
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target)

In [11]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target)

In [14]:
test_input.shape

(1625, 3)

In [16]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9975369458128078
0.8431855500821018


In [17]:
print(dt.score(test_input, test_target))

0.8436923076923077


### 교차검증
- `cross_validate` : 기본적으로 5-폴드 교차검증
    - 훈련 세트를 섞어서 폴드를 나누지 않음 => `cv=Stratified(Fold())` : 섞어서 나눔
    - `n_splits=` : 몇 폴드 교차검증을 할지

In [18]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00812292, 0.00564861, 0.00574279, 0.00559878, 0.00540662]), 'score_time': array([0.00389481, 0.00192976, 0.00209999, 0.0018599 , 0.00190306]), 'test_score': array([0.86358974, 0.87282051, 0.84702259, 0.86550308, 0.85934292])}


### 그리드 서치
- `GridSearchCV` : 하이퍼파라미터 탐색과 교차 검증을 한번에 수행
  - `n_jobs=` : 매개변수에서 병렬 실행에 사용할 CPU 코어 수 지정, 기본값 1,\
      -1로 지정하면 시스템에 있는 모든 코어 사용
- `params={}` : 그리드 서치 객체
    - `min_impurity_decrease` : 검증하고싶은 하이퍼 파라미터를 딕셔너리 키 값에 넣음
    - `max_depth` : 트리의 깊이 제한
    - `min_samples_split` : 노드를 나누기 위한 최소 샘플 수

In [25]:
from sklearn.model_selection import GridSearchCV

params = {
    'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004] # test해보고싶은 파라미터를 다 넣음
}

In [26]:
gs = GridSearchCV(dt, params, n_jobs=-1)

In [27]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004]})

In [28]:
gs.best_estimator_ # 어떤 값을 넣었더니 성능이 가장 좋았어?

DecisionTreeClassifier(min_impurity_decrease=0.0003)